In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

ModuleNotFoundError: No module named 'pandas'

In [ ]:
df = pd.read_csv("Algerian_forest_fires_dataset_cleaned.csv")
df.head()

In [ ]:
df.columns

In [ ]:
df.drop(["day","month","year"],axis = 1,inplace = True)

In [ ]:
df.head()

In [ ]:
df["Classes"] = np.where(df["Classes"].str.contains("not fire"),0,1)
df

In [ ]:
df["Classes"].value_counts()

In [ ]:
df.head()

In [ ]:
## independent and dependent features
x =df.drop(["FWI"],axis = 1)
y = df["FWI"]

In [ ]:
## trian test split
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.25,random_state = 42)

In [ ]:
## feature selection based on corr
x_train.corr()

In [ ]:
## check for multicollinearity
plt.figure(figsize = (12,10))
corr = x_train.corr()
sns.heatmap(corr,annot = True,cmap = "coolwarm")    

In [ ]:
def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold:
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

In [ ]:
## threshold -- setted by domain expert
corr_features = correlation(x_train,0.85)

In [ ]:
## drop these features
x_train.drop(corr_features,axis = 1,inplace = True)
x_test.drop(corr_features,axis = 1,inplace = True)

In [ ]:
x_train.shape, x_test.shape

In [ ]:
## Feature scaling or standardization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [ ]:
plt.subplots(figsize = (15,5))
plt.subplot(1,2,1)
sns.boxplot(data = x_train)
plt.title("x_train Before scaling")
plt.subplot(1,2,2)
sns.boxplot(data = X_train_scaled)
plt.title("x_train After scaling")

## linear regression

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x_train_scaled,y_train)

In [ ]:
y_pred = model.predict(x_test_scaled)


In [ ]:
from sklearn.metrics import r2_score
score = r2_score(y_test,y_pred)
print(score)

In [ ]:
plt.scatter(y_test,y_pred)

In [ ]:
## lasso regression
from sklearn.linear_model import Lasso
lasso = Lasso()
lasso.fit(x_train_scaled,y_train)
y_pred_lasso = lasso.predict(x_test_scaled)
score_lasso = r2_score(y_test,y_pred_lasso)
print(score_lasso)

In [ ]:
from sklearn.linear_model import LassoCV
lassocv = LassoCV(cv = 5)
lassocv.fit(x_train_scaled,y_train)

In [ ]:
lassocv.predict(x_test_scaled)

In [ ]:
lassocv.alpha_

In [ ]:
y_pred = lassocv.predict(x_test_scaled)
plt.scatter(y_test,y_pred)

In [ ]:
score1 = r2_score(y_test,y_pred)
print(score1)

In [ ]:
plt.scatter(y_test,y_pred_lasso)

In [ ]:
## ridge regression model
from sklearn.linear_model import Ridge
ridge = Ridge()
ridge.fit(x_train_scaled,y_train)
y_pred_ridge = ridge.predict(x_test_scaled)
score_ridge = r2_score(y_test,y_pred_ridge)
print(score_ridge)

In [ ]:
plt.scatter(y_test,y_pred_ridge)

In [ ]:
## Elasticnet
from sklearn.linear_model import ElasticNet
elasticnet = ElasticNet()
elasticnet.fit(x_train_scaled,y_train)
y_pred_elasticnet = elasticnet.predict(x_test_scaled)
score_elasticnet = r2_score(y_test,y_pred_elasticnet)
print(score_elasticnet)

In [ ]:
plt.scatter(y_test,y_pred_elasticnet)

In [ ]:
import pickle
pickle.dump(ridge,open("ridge.pkl","wb"))
pickle.dump(scaler,open("scaler.pkl","wb"))